In [102]:
import csv
import sqlparse
import sys
import re
import traceback
from collections import OrderedDict
from prettytable import PrettyTable
from collections import defaultdict
import pandas as pd

In [82]:
with open("metadata.txt",'r') as f:
        lines = f.readlines()
        print(lines)
        tableDict = {}
        for line in lines:
#             print(line)
            line = line.strip()
            print(line)
            if "<begin_table>" in line:
                flag = 1
                continue
            if flag == 1:
                tableName = line
                tableDict[tableName] = OrderedDict()
                flag = 0
            elif "<end_table>" not in line:
                tableDict[tableName][line] = []
        print(tableDict)

['<begin_table>\n', 'table1\n', 'A\n', 'B\n', 'C\n', '<end_table>\n', '<begin_table>\n', 'table2\n', 'B\n', 'D\n', '<end_table>\n']
<begin_table>
table1
A
B
C
<end_table>
<begin_table>
table2
B
D
<end_table>
{'table1': OrderedDict([('A', []), ('B', []), ('C', [])]), 'table2': OrderedDict([('B', []), ('D', [])])}


In [83]:
def read_table():
    tableDict = {}
    with open("metadata.txt",'r') as f:
            lines = f.readlines()
            print(lines)
            for line in lines:
    #             print(line)
                line = line.strip()
    #             print(line)
                if "<begin_table>" in line:
                    flag = 1
                    continue
                if flag == 1:
                    tableName = line
                    tableDict[tableName] = OrderedDict()
                    flag = 0
                elif "<end_table>" not in line:
                    tableDict[tableName][line] = []
            print(tableDict)
    return tableDict

In [84]:
tableDict = read_table()

['<begin_table>\n', 'table1\n', 'A\n', 'B\n', 'C\n', '<end_table>\n', '<begin_table>\n', 'table2\n', 'B\n', 'D\n', '<end_table>\n']
{'table1': OrderedDict([('A', []), ('B', []), ('C', [])]), 'table2': OrderedDict([('B', []), ('D', [])])}


In [85]:
# tablename = "table1"
def read_csv(tablename):
    tmp_dict = defaultdict(lambda : [])
    with open(tablename+".csv",'r') as f:
        for row in csv.reader(f,delimiter =','):
            for i,x in enumerate(row):
                tmp_dict[i].append(x)
    #             print(i,x)
#     print(tmp_dict)
    for idx,i in enumerate(tableDict[tablename].keys()):
        tableDict[tablename][i] = tmp_dict[idx]
    print(tableDict)

In [86]:
ta = tableDict.keys()
print(ta)
for i in ta:
    read_csv(i)

dict_keys(['table1', 'table2'])
{'table1': OrderedDict([('A', ['922', '640', '775', '-551', '-952', '-354', '-497', '411', '-900', '858']), ('B', ['158', '773', '85', '811', '311', '646', '335', '803', '718', '731']), ('C', ['5727', '5058', '10164', '1534', '1318', '7063', '4549', '10519', '9020', '3668'])]), 'table2': OrderedDict([('B', []), ('D', [])])}
{'table1': OrderedDict([('A', ['922', '640', '775', '-551', '-952', '-354', '-497', '411', '-900', '858']), ('B', ['158', '773', '85', '811', '311', '646', '335', '803', '718', '731']), ('C', ['5727', '5058', '10164', '1534', '1318', '7063', '4549', '10519', '9020', '3668'])]), 'table2': OrderedDict([('B', ['158', '773', '85', '811', '311', '646', '335', '803', '718', '731']), ('D', ['11191', '14421', '5117', '13393', '16116', '5403', '6309', '12262', '10226', '13021'])])}


In [206]:
def applyOp2(ls1, op, ls2):
    idxs = []
    print(ls1)
    print(ls2)
    if op == "=":
        for i in range(len(ls1)):
            if(ls1[i] == ls2[i]):
                idxs.append(i)
    elif op == ">":
        for i in range(len(ls1)):
            if(ls1[i] > ls2[i]):
                idxs.append(i)
    elif op == "<":
        for i in range(len(ls1)):
            if(ls1[i] < ls2[i]):
                idxs.append(i)
    elif op == "<=":
        for i in range(len(ls1)):
            if(ls1[i] <= ls2[i]):
                idxs.append(i)
    elif op == ">=":
        for i in range(len(ls1)):
            if(ls1[i] >= ls2[i]):
                idxs.append(i)
    else:
        raise N
    return idxs

In [210]:
def applyOp(cols, tables, colname, comp , value):
    curTable = tableDict[tables]
    print(curTable)
    
    ls1 = curTable[colname]
    print(ls1)
    ls1 = [int(i) for i in ls1]
    ls2 = [value] * len(ls1)
    idxs = applyOp2(ls1, comp, ls2)
    rows = []
    print(idxs)
    if idxs:
        for p in idxs:
            temp_row = []
            for q in cols:
                temp_row.append(curTable[q][p])
            rows.append(temp_row)
        df = pd.DataFrame(rows,columns=cols)
        print(df)
    else:
        df = pd.DataFrame([],columns=cols)
        print(df)

In [211]:
def processQuery(q):
    queries = sqlparse.parse(q)
    print(queries)
    gcols = []
    gtables = []
    for q in queries:
        distinct_flag = False
        print(q.tokens)
        if str(q.tokens[0]).lower() != "select":
            print("Not a valid query")
            continue
        i=2
        if str(q.tokens[i]).lower() == "distinct":
            distinct_flag = True
            i += 2
            
        if type(q.tokens[i]).__name__ == 'Identifier':
            cols = [str(q.tokens[i])]
        else:
            cols = list(q.tokens[i].get_identifiers())
            cols = [str(i) for i in cols]
        if str(q.tokens[i+2]).lower() != "from":
            print("Invalid query")
            continue
        if type(q.tokens[i+4]).__name__ == 'Identifier':
            tables = str(q.tokens[i+4])
        else:
            tables = list((q.tokens[i+4]).get_identifiers())
            tables = [str(i) for i in tables]
        i=6
#         print(len(q.tokens))
        if len(q.tokens) > i+1:
            whereTokens = q.tokens[i+2].tokens
            whereTokens = whereTokens[2:]
            print(whereTokens)
            for cond in whereTokens:
                if type(cond).__name__ == "Comparison":
                    colname = ""
                    comp = ""
                    value = 0
                    tkns = cond.tokens
                    print(tkns)
                    for tk in tkns:
                        print(tk)
                        if type(tk).__name__ == "Identifier":
                            colname = str(tk)
                        if str(tk.ttype) == "Token.Operator.Comparison":
                            comp = str(tk)
                        if str(tk.ttype).startswith('Token.Literal'):
                            value = int(str(tk))
            print(colname, comp, value)
            applyOp(cols, tables, colname, comp , value)
#         print(tables)
#         print(cols)
        gcols.append(cols)
        gtables.append(tables)
    return gcols, gtables    

In [212]:
processQuery("Select A from table1 where A > 500")#; Select B,D from table2")

(<Statement 'Select...' at 0x7F0C8F2240C0>,)
[<DML 'Select' at 0x7F0C8F225B88>, <Whitespace ' ' at 0x7F0C8F419948>, <Identifier 'A' at 0x7F0C8F224048>, <Whitespace ' ' at 0x7F0C8F4198E8>, <Keyword 'from' at 0x7F0C8F23E828>, <Whitespace ' ' at 0x7F0C8F23EE28>, <Identifier 'table1' at 0x7F0C8F224390>, <Whitespace ' ' at 0x7F0C8F23EEE8>, <Where 'where ...' at 0x7F0C8F2242A0>]
[<Comparison 'A > 500' at 0x7F0C8F2244F8>]
[<Identifier 'A' at 0x7F0C8F2241B0>, <Whitespace ' ' at 0x7F0C8F2300A8>, <Comparison '>' at 0x7F0C8F230108>, <Whitespace ' ' at 0x7F0C8F230168>, <Integer '500' at 0x7F0C8F2301C8>]
A
 
>
 
500
A > 500
OrderedDict([('A', ['922', '640', '775', '-551', '-952', '-354', '-497', '411', '-900', '858']), ('B', ['158', '773', '85', '811', '311', '646', '335', '803', '718', '731']), ('C', ['5727', '5058', '10164', '1534', '1318', '7063', '4549', '10519', '9020', '3668'])])
['922', '640', '775', '-551', '-952', '-354', '-497', '411', '-900', '858']
[922, 640, 775, -551, -952, -354, -497

([['A']], ['table1'])

In [213]:
nqueries = len(gcols)

In [197]:
for i in range(nqueries):
    cols = gcols[i]
    tables = gtables[i]
    
    rows = []
#     rows.append(cols)
    
    temp_array = []
    for j in cols:
        temp_array.append(tableDict[tables][j])
        
    elem = len(temp_array[0])
    ln = len(temp_array)
    for k in range(elem):
        temp_row = []
        for j in range(ln):
            temp_row.append(temp_array[j][k])
        rows.append(temp_row)
    df = pd.DataFrame(rows,columns=cols)
    print(df)

      A
0   922
1   640
2   775
3  -551
4  -952
5  -354
6  -497
7   411
8  -900
9   858
     B      D
0  158  11191
1  773  14421
2   85   5117
3  811  13393
4  311  16116
5  646   5403
6  335   6309
7  803  12262
8  718  10226
9  731  13021
